In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random
import glob

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Encoder(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.enc1 = DoubleConv(in_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        return enc1, enc2, enc3, enc4

class Bottleneck(nn.Module):
    def __init__(self):
        super().__init__()
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.bottleneck(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.upconv4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)

    def forward(self, bottleneck, skip1, skip2, skip3, skip4):
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat([dec4, skip4], dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat([dec3, skip3], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, skip2], dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, skip1], dim=1)
        dec1 = self.dec1(dec1)
        return dec1

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):  # Ubah ke 1 channel
        super().__init__()
        self.encoder = Encoder(in_channels)
        self.bottleneck = Bottleneck()
        self.decoder = Decoder()
        self.final_conv = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        skip1, skip2, skip3, skip4 = self.encoder(x)
        bottleneck = self.bottleneck(skip4)
        decoded = self.decoder(bottleneck, skip1, skip2, skip3, skip4)
        return self.final_conv(decoded)

In [ ]:
# Custom Dataset
class SatelliteDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = glob.glob(os.path.join(image_dir, "*.jpg")) + \
                     glob.glob(os.path.join(image_dir, "*.png"))

    def add_noise(self, image):
        # Menambahkan berbagai jenis noise
        noise_type = random.choice(['gaussian', 'salt_pepper', 'speckle'])

        if noise_type == 'gaussian':
            # Gaussian noise
            mean = 0
            sigma = random.uniform(0.02, 0.1)
            noise = torch.normal(mean, sigma, image.shape)
            noisy_image = image + noise

        elif noise_type == 'salt_pepper':
            # Salt and pepper noise
            prob = random.uniform(0.02, 0.1)
            noise = torch.rand(image.shape)
            salt = (noise > 1 - prob/2).float()
            pepper = (noise < prob/2).float()
            noisy_image = image * (1 - salt - pepper) + salt

        else:  # Speckle
            # Speckle noise
            intensity = random.uniform(0.05, 0.2)
            noise = torch.randn(image.shape) * intensity
            noisy_image = image + image * noise

        # Clip values to valid range [0, 1]
        return torch.clamp(noisy_image, 0, 1)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        # Baca gambar sebagai grayscale langsung
        image = Image.open(img_path).convert('L')

        if self.transform:
            image = self.transform(image)

        # Original image sebagai target (clean)
        target = image.clone()
        # Menambahkan noise pada input image
        noisy_image = self.add_noise(image)

        # Menambah dimensi channel (1 channel untuk grayscale)
        noisy_image = noisy_image.unsqueeze(0) if noisy_image.dim() == 2 else noisy_image
        target = target.unsqueeze(0) if target.dim() == 2 else target

        return noisy_image, target

In [ ]:
# Cek dataset in drive
train_dir = '/content/drive/MyDrive/Dataset/satellit_dataset/Train'
valid_dir = '/content/drive/MyDrive/Dataset/satellit_dataset/Val'

def check_directory(dir_path):
    if os.path.exists(dir_path):
        print(f"Direktori '{dir_path}' ada.")
    else:
        print(f"Direktori '{dir_path}' tidak ada.")

check_directory(train_dir)
check_directory(valid_dir)

Direktori '/content/drive/MyDrive/Dataset/satellit_dataset/Train' ada.
Direktori '/content/drive/MyDrive/Dataset/satellit_dataset/Val' ada.


In [ ]:
# Training function dengan visualisasi hasil
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs, device, save_dir):
    history = {
        'train_loss': [],
        'valid_loss': [],
    }

    best_valid_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Training]')

        for batch_idx, (noisy_imgs, target_imgs) in enumerate(train_pbar):
            noisy_imgs, target_imgs = noisy_imgs.to(device), target_imgs.to(device)

            optimizer.zero_grad()
            outputs = model(noisy_imgs)
            loss = criterion(outputs, target_imgs)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_pbar.set_postfix({'loss': loss.item()})

            # Visualisasi hasil setiap 100 batch
            if batch_idx % 100 == 0:
                with torch.no_grad():
                    # Ambil satu gambar dari batch
                    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

                    # Plot noisy input
                    axes[0].imshow(noisy_imgs[0].cpu().squeeze(), cmap='gray')
                    axes[0].set_title('Noisy Input')
                    axes[0].axis('off')

                    # Plot clean target
                    axes[1].imshow(target_imgs[0].cpu().squeeze(), cmap='gray')
                    axes[1].set_title('Clean Target')
                    axes[1].axis('off')

                    # Plot prediction
                    axes[2].imshow(outputs[0].cpu().detach().squeeze(), cmap='gray')
                    axes[2].set_title('Model Prediction')
                    axes[2].axis('off')

                    plt.savefig(os.path.join(save_dir, f'progress_epoch{epoch+1}_batch{batch_idx}.png'))
                    plt.close()

        avg_train_loss = train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        # Validation phase
        model.eval()
        valid_loss = 0
        valid_pbar = tqdm(valid_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validation]')

        with torch.no_grad():
            for batch_idx, (noisy_imgs, target_imgs) in enumerate(valid_pbar):
                noisy_imgs, target_imgs = noisy_imgs.to(device), target_imgs.to(device)

                outputs = model(noisy_imgs)
                loss = criterion(outputs, target_imgs)

                valid_loss += loss.item()
                valid_pbar.set_postfix({'loss': loss.item()})

        avg_valid_loss = valid_loss / len(valid_loader)
        history['valid_loss'].append(avg_valid_loss)

        print(f'\nEpoch {epoch+1}/{num_epochs}:')
        print(f'Average Training Loss: {avg_train_loss:.4f}')
        print(f'Average Validation Loss: {avg_valid_loss:.4f}')

        # Save best model
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'valid_loss': avg_valid_loss,
            }, os.path.join(save_dir, 'best_model.pth'))
            print(f'Model saved at epoch {epoch+1}')

        # Plot learning curves
        plt.figure(figsize=(10, 5))
        plt.plot(history['train_loss'], label='Training Loss')
        plt.plot(history['valid_loss'], label='Validation Loss')
        plt.title('Learning Curves')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(save_dir, 'learning_curves.png'))
        plt.close()

    return history

# Main training pipeline
def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Hyperparameters
    BATCH_SIZE = 8
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.001
    IMAGE_SIZE = 256

    # Transform
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
    ])

    # Create datasets
    train_dataset = SatelliteDataset(
        image_dir='/content/drive/MyDrive/Dataset/satellit_dataset/Train',
        transform=transform
    )

    valid_dataset = SatelliteDataset(
        image_dir='/content/drive/MyDrive/Dataset/satellit_dataset/Val',
        transform=transform
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2
    )

    # Initialize model, criterion, and optimizer
    model = UNet(in_channels=1, out_channels=1).to(device)  # Ubah ke 1 channel
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Create directory for saving model and plots
    save_dir = '/content/drive/MyDrive/Dataset/satellit_dataset/model_saves'
    os.makedirs(save_dir, exist_ok=True)

    # Train model
    history = train_model(
        model=model,
        train_loader=train_loader,
        valid_loader=valid_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=NUM_EPOCHS,
        device=device,
        save_dir=save_dir
    )

    print("Training completed!")

if __name__ == "__main__":
    main()

Using device: cuda


Epoch 1/50 [Validation]:  89%|████████▉ | 223/250 [09:44<01:10,  2.60s/it, loss=0.00149]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94720000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 1/50 [Validation]: 100%|██████████| 250/250 [10:57<00:00,  2.63s/it, loss=0.00104]



Epoch 1/50:
Average Training Loss: 0.0049
Average Validation Loss: 0.0015
Model saved at epoch 1


Epoch 2/50 [Validation]:  89%|████████▉ | 223/250 [00:15<00:01, 18.81it/s, loss=0.00113]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94720000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 2/50 [Validation]: 100%|██████████| 250/250 [00:19<00:00, 12.96it/s, loss=0.000779]



Epoch 2/50:
Average Training Loss: 0.0030
Average Validation Loss: 0.0012
Model saved at epoch 2


...


Epoch 48/50:
Average Training Loss: 0.0006
Average Validation Loss: 0.0006


Epoch 49/50 [Validation]:  89%|████████▉ | 223/250 [00:15<00:01, 18.90it/s, loss=0.000389]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94720000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 49/50 [Validation]: 100%|██████████| 250/250 [00:19<00:00, 13.04it/s, loss=0.000412]



Epoch 49/50:
Average Training Loss: 0.0006
Average Validation Loss: 0.0006


Epoch 50/50 [Validation]:  89%|████████▉ | 222/250 [00:15<00:01, 17.92it/s, loss=0.000713]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94720000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 50/50 [Validation]: 100%|██████████| 250/250 [00:19<00:00, 12.92it/s, loss=0.000407]



Epoch 50/50:
Average Training Loss: 0.0006
Average Validation Loss: 0.0006
Training completed!
